##Installation

In [1]:
#Forking text from newspaper
!pip3 install newspaper3k

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 10.9 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=e1e855588abd6308025ffcc33df4c8088c947af383ccb896c7b4c88d6fdb654a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/f8/cce3a9ae6d828bd346be695f7ff54612cd22b7cbd7208d68f3
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=d2f025458dd33a14f309dd896feda4806125f6a1b81989234d9aa336dd465b1c
  Stored in directory: /root/.cache/pip/wheels/80/d5/72/9cd9eccc819636436c6a6e59c22a0fb1e

In [2]:
#additional lib for the newspaper lib
!pip install "lxml[html_clean]"

In [3]:
#sentence tokenizer
!pip install nltk

In [4]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
#question generate
!pip install transformers sentencepiece

In [ ]:

#quick test
from transformers import pipeline

# Load the question generation pipeline
qg = pipeline("text2text-generation", model="valhalla/t5-small-qg-prepend")

sentence = "Marie Curie discovered radium in 1898."
input_text = "generate question: " + sentence

# Generate the question
result = qg(input_text)
print(result[0]['generated_text'])  # e.g., "Who discovered radium in 1898?"

In [ ]:
from transformers import pipeline

# Load the question answering pipeline
qa = pipeline("question-answering")

# Example context (the text you want to find answers from)
context = "Albert Einstein was a theoretical physicist who developed the theory of relativity."

# Example question (you could generate this using your question generator)
question = "Who developed the theory of relativity?"

# Use the QA pipeline to find the answer
result = qa(question=question, context=context)

print("Answer:", result['answer'])

##Testing


In [7]:
from newspaper import Article

url = 'https://yle.fi/a/74-20174760'
article = Article(url, language='en')
article.download()
article.parse()

In [8]:
print(article.title)

Finland sees 30-degree temps for 17th day in a row


In [9]:
print(article.text)

The ongoing heatwave shows no sign of dissipating anytime soon.

More heat records could be broken in the coming days, according to Yle meteorologist Elias Paakkanen.

Finland saw another heat record broken on Monday, as the temperature breached the 30-degree-Celsius mark somewhere in the country for the 17th consecutive day.

This time, the record was broken in the municipality of Lieksa, in North Karelia, where it reached 30.1 degrees at just after 5 pm.

As the ongoing heatwave shows no sign of dissipating anytime soon, more heat records could be broken in the coming days, according to Yle meteorologist Elias Paakkanen.


In [10]:
from nltk.tokenize import sent_tokenize

In [11]:
# Split into sentences
sentences = sent_tokenize(article.text, language='finnish')

In [12]:
for i, sentence in enumerate(sentences[:5]):
    print(f"{i+1}. {sentence}")

1. The ongoing heatwave shows no sign of dissipating anytime soon.
2. More heat records could be broken in the coming days, according to Yle meteorologist Elias Paakkanen.
3. Finland saw another heat record broken on Monday, as the temperature breached the 30-degree-Celsius mark somewhere in the country for the 17th consecutive day.
4. This time, the record was broken in the municipality of Lieksa, in North Karelia, where it reached 30.1 degrees at just after 5 pm.
5. As the ongoing heatwave shows no sign of dissipating anytime soon, more heat records could be broken in the coming days, according to Yle meteorologist Elias Paakkanen.


In [13]:
type(sentences)

list

In [24]:
#make english question
#question: More heat records could be broken in the coming days, according to Yle meteorologist Elias Paakkanen.
from transformers import pipeline

# Load the question generation pipeline
qg = pipeline("text2text-generation", model="valhalla/t5-small-qg-prepend")

sentence = sentences[1]
# A clear and focused prompt to guide question generation
input_text = (
    "generate a question focused on what the person predicts or thinks, "
    "using a clause about the prediction or opinion, not copying the whole sentence, "
    "for example (do not output), 'Albert Einstein predicted gravity' -> 'What did Albert Einstein predict?' "
    "make question that less than half the length of the statement"
    + sentence
)

# Generate the question
result_question = qg(input_text)
print(result_question[0]['generated_text'])

Device set to use cuda:0


What could be broken in the coming days?


In [25]:
from transformers import pipeline

# Load the question answering pipeline
qa = pipeline("question-answering", model="deepset/roberta-base-squad2")

# Example context (the text you want to find answers from)
context = sentences[1]

# Example question (you could generate this using your question generator)
question = result_question[0]['generated_text']

# Use the QA pipeline to find the answer
result = qa(question=question, context=context)

print("Answer:", result['answer'])

Device set to use cuda:0


Answer: heat records


In [27]:
print(result_question[0]['generated_text'])
user_answer = input("Your answer: ")
if user_answer == result['answer']:
    print("Correct!")
else:
    print("Incorrect!")

What could be broken in the coming days?
Your answer: heat records
Correct!
